<h1>Start by activating our project enviornment and installing required dependencies</h1>

In [1]:
using Pkg
Pkg.activate("RadarQC")

##Make sure Julia can see our module 
push!(LOAD_PATH, "./src/")

###Load key functionality 
###This will take a while the first time you do it 
using RadarQC

  Activating project at `~/Documents/Grad_School/Research/RadarQC/RadarQC`
[ Info: Precompiling RadarQC [top-level]


<h2>Let's begin by splitting our CFRadials into a Training and a Testing directory</h2>
<h2><span style="color:Red">WARNING: This function will begin by DELETING the training and testing directories to clean them</span></h2>

In [11]:
###Make sure to use absolute paths here 
CASE_PATHS= ["/Users/ischluesche/Documents/Grad_School/Research/RadarQC/CFRADIALS/CASES/BAMEX", 
             "/Users/ischluesche/Documents/Grad_School/Research/RadarQC/CFRADIALS/CASES/HAGUPIT", 
             "/Users/ischluesche/Documents/Grad_School/Research/RadarQC/CFRADIALS/CASES/RITA", 
             "/Users/ischluesche/Documents/Grad_School/Research/RadarQC/CFRADIALS/CASES/VORTEX"]

TRAINING_PATH = "/Users/ischluesche/Documents/Grad_School/Research/RadarQC/CFRADIALS/CASES/TRAINING"
TESTING_PATH = "/Users/ischluesche/Documents/Grad_School/Research/RadarQC/CFRADIALS/CASES/TESTING"

"/Users/ischluesche/Documents/Grad_School/Research/RadarQC/CFRADIALS/CASES/TESTING"

In [12]:

split_training_testing!(CASE_PATHS, TRAINING_PATH, TESTING_PATH)


TOTAL NUMBER OF TDR SCANS ACROSS ALL CASES: 1780
TESTING SCANS PER CASE 89
NUMBER OF SCANS IN CASE: 482
TRAINING GROUP SIZE: 196 + REMAINDER: 1
TESTING GROUP SIZE: 29 + REMAINDER 2

 INDEXES 1 TO 31 ASSIGNED TESTING
 INDEXES 31  TO 228 ASSIGNED TRAINING
 INDEXES 229 TO 257 ASSIGNED TESTING
 INDEXES 258 TO 453 ASSIGNED TRAINING
 INDEXES 454 TO 482 ASSIGNED TESTING
Total length of case files: 482
Length of testing files: 89 - 0.18464730290456433 percent
Length of testing_files: 393 - 0.8153526970954357 percent
NUMBER OF SCANS IN CASE: 238
TRAINING GROUP SIZE: 74 + REMAINDER: 1
TESTING GROUP SIZE: 29 + REMAINDER 2

 INDEXES 1 TO 31 ASSIGNED TESTING
 INDEXES 31  TO 106 ASSIGNED TRAINING
 INDEXES 107 TO 135 ASSIGNED TESTING
 INDEXES 136 TO 209 ASSIGNED TRAINING
 INDEXES 210 TO 238 ASSIGNED TESTING
Total length of case files: 238
Length of testing files: 89 - 0.3739495798319328 percent
Length of testing_files: 149 - 0.6260504201680672 percent
NUMBER OF SCANS IN CASE: 794
TRAINING GROUP SIZE

<h2>Now, calculate input features based off of the training/testing data<h2> 
<h1><span style="color:Green">This takes up a lot of memory in a notebook, so I would recommend running this in a script</span></h1>

In [2]:
###Arguments are
###Path to training file/directory 
###Path to file containing arguments on features to calculate 
###Where to output calculated features 

###This will use the 5-parameter model described in the referenced manuscript 

###I have specified the verbose=false flag so as to not overload notebook memory 
###Generally diagnostics about timing will print out 
calculate_features("./CFRADIALS/CASES/TRAINING", "./MODELS/DesRosiers_Bell_23/config.txt", "training_features.h5"; verbose=false)
calculate_features("./CFRADIALS/CASES/TESTING", "./MODELS/DesRosiers_Bell_23/config.txt", "testing_features.h5"; verbose=false)

OUTPUTTING DATA IN HDF5 FORMAT TO FILE: training_features.h5
  1.340009 seconds (6.02 M allocations: 415.260 MiB, 8.55% gc time)
  0.118406 seconds (1.47 M allocations: 131.398 MiB, 19.58% gc time)
  0.108421 seconds (1.46 M allocations: 131.706 MiB, 9.78% gc time)
  0.107725 seconds (1.47 M allocations: 131.900 MiB, 9.80% gc time)
  0.105562 seconds (1.46 M allocations: 131.748 MiB, 9.70% gc time)
  0.106566 seconds (1.46 M allocations: 131.416 MiB, 9.41% gc time)
  0.106803 seconds (1.47 M allocations: 132.227 MiB, 8.67% gc time)
  0.116056 seconds (1.47 M allocations: 131.935 MiB, 12.73% gc time)
  0.110938 seconds (1.47 M allocations: 132.213 MiB, 11.32% gc time)
  0.103943 seconds (1.47 M allocations: 131.917 MiB, 6.49% gc time)
  0.103725 seconds (1.46 M allocations: 131.642 MiB, 5.96% gc time)
  0.103594 seconds (1.46 M allocations: 131.436 MiB, 6.33% gc time)
  0.103307 seconds (1.46 M allocations: 131.674 MiB, 6.34% gc time)
  0.105206 seconds (1.46 M allocations: 131.498 MiB,

POSSIBLE ERRONEOUS CFRAD DIMENSIONS... SKIPPING ./CFRADIALS/CASES/TRAINING/cfrad.20050922_180849.988_to_20050922_180849.988_TF-ELDR_AIR.nc


  0.220430 seconds (1.57 M allocations: 141.221 MiB, 34.96% gc time)
  0.192244 seconds (1.57 M allocations: 141.020 MiB, 20.97% gc time)
  0.975566 seconds (1.57 M allocations: 140.580 MiB, 85.86% gc time)
  0.111131 seconds (1.56 M allocations: 140.070 MiB, 7.01% gc time)
  0.115439 seconds (1.56 M allocations: 140.222 MiB, 7.14% gc time)
  0.149018 seconds (1.57 M allocations: 140.639 MiB, 15.18% gc time)
  0.154933 seconds (1.57 M allocations: 140.706 MiB, 14.50% gc time)
  0.926650 seconds (1.57 M allocations: 140.627 MiB, 87.55% gc time)
  0.114029 seconds (1.57 M allocations: 140.715 MiB, 6.51% gc time)
  0.108799 seconds (1.57 M allocations: 140.623 MiB, 6.61% gc time)
  0.111382 seconds (1.57 M allocations: 140.716 MiB, 6.85% gc time)
  0.210959 seconds (1.57 M allocations: 140.642 MiB, 26.61% gc time)
  1.151756 seconds (1.57 M allocations: 140.723 MiB, 84.67% gc time)
  0.110930 seconds (1.57 M allocations: 140.612 MiB, 6.68% gc time)
  0.120513 seconds (1.57 M allocations: 

LoadError: InterruptException:

In [33]:
###Now, let's train our model on it! 
RadarQC.train_model("training_features.h5", "trained_model.joblib"; verify=true)

PyCall.PyError: PyError ($(Expr(:escape, :(ccall(#= /Users/ischluesche/.julia/packages/PyCall/gaz9g/src/pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'ValueError'>
ValueError('Found input variables with inconsistent numbers of samples: [22037873, 482]')
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/sklearn/base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/sklearn/ensemble/_forest.py", line 348, in fit
    X, y = self._validate_data(
           ^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/sklearn/base.py", line 622, in _validate_data
    X, y = check_X_y(X, y, **check_params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/sklearn/

<h1>Now, select variables you wish to QC and apply the model to them!</h1> 

In [ ]:
###These need to have the same name as in the cfradial file
VARS_TO_QC = ["ZZ", "VV"]

QC_scan("/Users/ischluesche/Documents/Grad_School/Research/JMLQC/CFRADIALS/SMALL_TEST/cfrad.19950516_221950.411_to_19950516_221953.219_TA-ELDR_AIR.nc", "sample_config.txt", "trained_model.bson"; 
            VARIABLES_TO_QC = VARS_TO_QC)